In [ ]:
import cv2
import glob
import io
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display
from birds import BirdsImage

In [ ]:
image_files = list(glob.glob('photos/*'))

# UI
dropdown = widgets.Dropdown(
    options=image_files,
    value=image_files[0],
    description='Photo:',
)
slider = widgets.IntSlider(
    value=127,
    min=0,
    max=255,
    step=1,
    description='Threshold:',
    orientation='horizontal',
)
label = widgets.Label()
image = widgets.Image(value=b'', format='jpg')
grid = widgets.GridBox(
    children=[dropdown, slider, label, image], 
    layout=widgets.Layout(grid_template_rows='auto')
)

def render(path, thresh):
    birds = BirdsImage(filename=path, thresh=thresh)
    birds.fit()
    side_by_side = birds.side_by_side()
    format_ = Path(path).suffix

    return (birds.count, cv2.imencode(format_, side_by_side)[1].tobytes())

def on_user_input(change):
    cnt, img = render(dropdown.value, int(slider.value))
    birds = 'bird' if cnt == 1 else 'birds'
    label.value = f'Counted: {cnt} {birds} [thresh={slider.value}]'
    image.value = img

slider.observe(on_user_input)
dropdown.observe(on_user_input)
on_user_input(None)

display(grid)